In [204]:
import importlib
import SpotifyAPIClient
import pprint
import csv
import os
import random
importlib.reload(SpotifyAPIClient)
from SpotifyAPIClient import SpotifyAPI
import pandas as pd

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import mysklearn.myutils as myutils

import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyNaiveBayesClassifier


client_id = 'd7a2e6f4a8434550baa5eda073f0a6a3'
client_secret = 'def46d47ba584e378b7667645666a468'
playlist_id = '7L736vCRhBe5EapwwkutUl'

In [205]:
spotify = SpotifyAPI(client_id, client_secret)
response = spotify.get_playlist(playlist_id)


In [206]:
track_ids, track_popularity = spotify.parse_for_track_id(response)
#print("track_ids: ", track_ids)
print()
print()
header = ["Track Name", "Track ID", "Danceability", "Energy", "Valence", "Popularity"] 
tracks_feature_list = []

did_once = False
count = 0
features_header = []
features_header.append("song_id")
features_header.append("song_name")
songs_vals_list = [] #[song_id, song_name, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, type, id, uri, track_href, analysis_url, duration_ms, time_signature ]

for name, id in track_ids.items():
    #print("name: ", name, " ----- id: ", id)
    song = []
    song.append(id)
    song.append(name)

    track_feat_list = []
    response = spotify.get_track_audio_features(id)
    #print("response: ", response)
    do_not_add = ["type","id","uri","track_href","analysis_url"]

    for k, v in response.items():
        #print("     k: ", k, "      v: ", v)
        #type,id,uri,track_href,analysis_url
        if(k not in do_not_add):
            if not did_once:
                features_header.append(k)
                #print("v type: ", type(v))
            song.append(v)
    did_once = True
    song.append(track_popularity[count])
    songs_vals_list.append(song)

    track_feat_list.append(name)
    track_feat_list.append(id)
    track_feat_list.append(round(response['danceability'] * 100, 2))
    track_feat_list.append(round(response['energy'] * 100, 2))
    # track_feat_list.append(response['tempo'])
    track_feat_list.append(round(response['valence'] * 100, 2))
    #print("track_popularity[count]: ", track_popularity[count])
    track_feat_list.append(track_popularity[count])
    count += 1
    tracks_feature_list.append(track_feat_list)
features_header.append("song_popularity")

#print("features_header: ", features_header)
#print("songs_vals_list: ", songs_vals_list)
#myutils.table_val_print(features_header, songs_vals_list)
#myutils.print_tabulate(songs_vals_list, features_header)
myutils.print_tabulate(tracks_feature_list, header)


    



====================================================  ======================  ==============  ========  =========  ============
Track Name                                            Track ID                  Danceability    Energy    Valence    Popularity
====================================================  ======================  ==============  ========  =========  ============
Are We There Yet                                      39LRXm8xZ3DP2W37sl1NDj            71.1      58.1       62.1            43
Sweet Nothing (feat. Florence Welch)                  24LS4lQShWyixJ0ZrJXfJ5            57.3      92.9       58.2            73
Wish You Were Mine - Radio Edit                       412luShbmlgqqgYFStIB1s            65.4      83.2       40.7            66
All My Love (feat. Conor Maynard)                     1839RWzPRZCVgvamu4sHSN            59        87.1       30.4            60
Closer                                                7BKLCZ1jbUBVqRi2FVlTVw            74.8      52.4

In [207]:
file_name = os.path.join('data', 'key-track-audio-features.txt')
with open(file_name, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(header)
    csv_writer.writerows(tracks_feature_list)

file_name2 = os.path.join('data', 'track-audio-features-all.txt')
with open(file_name2, 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(features_header)
    csv_writer.writerows(songs_vals_list)

In [208]:
# Get the file data
fname = os.path.join("data", "track-audio-features-all.txt")
track_data = MyPyTable().load_from_file(fname)

print("===========================================")
print("Nearest Neighbor 'Popularity' Classifier (k=5)")
print("===========================================")

mknc = MyKNeighborsClassifier(5) # set the amount of neighbors to 5

# Get the 3 column data sets
danceability = track_data.get_column('danceability') 
energy = track_data.get_column('energy')
valence = track_data.get_column('valence')

# store each columns data in x_vals and scale the data
x_vals = [[danceability[i], energy[i],valence[i]] for i in range(len(danceability))]
print()
print("x_vals: ", x_vals)
#scaled_x, blank = myutils.scale(x_vals, [[]])

# train and split the test list
x_train_split, x_test_split, y_train_split, y_test_split = myevaluation.train_test_split(scaled_x, 
track_data.get_column('song_popularity'), shuffle=True)
print()
print()
print("x_train_split: ",  x_train_split) 
print()
print("x_test_split: ", x_test_split) 
print()
print("y_train_split: ", y_train_split)
print()
print("y_test_split: ", y_test_split)

mknc.fit(x_train_split, track_data.get_column('song_popularity')) # fit the data
rand_rows = myutils.get_rand_rows(track_data, 5) # randomize 5 rows

# predict using the regression line
predicted = mknc.predict(x_test_split)
print("predicted: ", predicted)
# predicted = [jupyter_utils.get_rating(val) for val in predicted]
# expected = [jupyter_utils.get_rating(val) for val in y_test_split]

# # for each row in the random list of 5
# for row in rand_rows:
#     print('instance:', row)
#     print("row[4]: ", row[2])
#     print("row[1]: ", row[3])
#     print("row[5]: ", row[11])
#     scaled_vals, scaled_test = myutils.scale(x_vals, [[row[2],row[3],row[11]]])
#     print("scaled_vals: ", scaled_vals)
#     print("scaled_test: ", scaled_test)
#     #print('class:', myutils.get_rating(mknc.predict(scaled_test)[0]), 'actual: ', juputils.get_rating(row[0]))

Nearest Neighbor 'Popularity' Classifier (k=5)

x_vals:  [[0.711, 0.581, 0.621], [0.573, 0.929, 0.582], [0.654, 0.832, 0.407], [0.59, 0.871, 0.304], [0.748, 0.524, 0.661], [0.584, 0.712, 0.286], [0.742, 0.838, 0.694], [0.78, 0.798, 0.771], [0.388, 0.759, 0.286], [0.757, 0.711, 0.649], [0.506, 0.805, 0.383], [0.321, 0.675, 0.274], [0.596, 0.854, 0.152], [0.738, 0.814, 0.483], [0.784, 0.757, 0.587], [0.61, 0.841, 0.431], [0.664, 0.708, 0.742], [0.646, 0.823, 0.418], [0.788, 0.604, 0.16], [0.753, 0.717, 0.859], [0.715, 0.581, 0.378], [0.701, 0.695, 0.624], [0.737, 0.667, 0.658], [0.733, 0.71, 0.965], [0.664, 0.705, 0.477], [0.578, 0.862, 0.547], [0.719, 0.84, 0.636], [0.818, 0.705, 0.772], [0.494, 0.842, 0.574], [0.43, 0.392, 0.127], [0.766, 0.674, 0.214], [0.803, 0.499, 0.749], [0.67, 0.855, 0.648], [0.687, 0.71, 0.886], [0.442, 0.83, 0.578], [0.67, 0.668, 0.608], [0.849, 0.483, 0.282], [0.715, 0.635, 0.39], [0.629, 0.828, 0.512], [0.674, 0.74, 0.431], [0.64, 0.648, 0.548], [0.674, 0.774